In [1]:
"""Combining Front Office and Middle Office

Demonstration of how to give front office users an view of intraday trading activity on top of their middle office IBOR.

Attributes
----------
instruments
transactions
sub-holdings keys
properties
orders
derived portfolios
cocoon
"""

'Combining Front Office and Middle Office\n\nDemonstration of how to give front office users an view of intraday trading activity on top of their middle office IBOR.\n\nAttributes\n----------\ninstruments\ntransactions\nsub-holdings keys\nproperties\norders\nderived portfolios\ncocoon\n'

# Combining Front Office and Middle Office

In this example we demonstrate how to give front office users a view of intraday trading activity on top of their middle office IBOR. We set up a portfolio with multiple strategies and generate orders which are booked back into a [derived portfolio](https://support.finbourne.com/what-is-a-derived-portfolio) showing updated positions in the strategies.

To illustrate, below we will BUY and SELL <i>Amazon, Inc.</i> stock across two strategies, and show an updated view of the portfolio intraday. Consider the <b>Tech</b> strategy. At SOD we have 5000 units, we SELL 500 units, leaving 4500 in the portfolio. 

<br>


|Portfolio Code | Instrument Name |ISIN| Strategy  | SOD Holdings | Intraday orders | Intraday Holdings|
|:-------------:| :-------------: | :-----------------: |:-------------: | :-------------: | :-------------: | :-------------: |
|us_long_live| Amazon |US0231351067 |Balanced | 0 | BUY of 4000 units | <font color="red">4000 </font>|
|us_long_live| Amazon |US0231351067|Tech | 5000 | SELL of 500 units | <font color="red">4500 </font>|

<br>

## Imports

In [2]:
import lusid
import lusid.models as models
import lusid.api as la
import lusid.models as lm
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)

from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import os

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.0.1.0


Define scope, portfolio and some other variables used in the example:

In [3]:
orders_scope = "combining_fo_and_mo_scope"
orders_portfolio = "us_long_live"
ibor_scope = "middle_office"
ibor_portfolio = "us_long"
strategy_shk = "strategy"
executed_price_code = "executed_price"
execution_price_property = f"Allocation/{orders_scope}/{executed_price_code}"
allocation_strategy_property = f"Allocation/{orders_scope}/{strategy_shk}"
pf_created_date = "2020-01-01T00:00:00+00:00"

## Load an Instrument Master into LUSID

In [4]:
instr_df = pd.read_csv("data/multiplecurrencies-instruments.csv")
display(instr_df)

instrument_name client_internal currency          isin  \
0          Amazon_Nasdaq_AMZN    imd_34634534      USD  US0231351067   
1           Apple_Nasdaq_AAPL    imd_35345345      USD  US0378331005   
2        USTreasury_2.00_2021    imd_34535347      USD  US912828U816   
3       USTreasury_6.875_2025    imd_34534539      USD  US912810EV62   
4    ExpressScripts_NYSE_ESRX    imd_34352311      USD  US30219G1085   
5  TrinityIndustries_NYSE_TRN    imd_34235200      USD  US8965221091   
6              Trex_NYSE_TREX    imd_32423956      USD  US89531P1057   
7               Cigna_NYSE_CI    imd_32452391      USD  US1255091092   
8             Arcosa_NYSE_ACA    imd_23423409      USD  US0396531008   

           figi exchange_code          country_issue            ticker  \
0  BBG000BVPXP1            UN  united_states_america              AMZN   
1  BBG000B9XVV8            UN  united_states_america              AAPL   
2  BBG00FN3B5K8        BERLIN  united_states_america      T 2 12/31/21   
3  BBG000DQQNJ8      NEW YORK  united_states_america  T 6.875 08/15/25   
4  BBG000C16621            UN  united_states_america              ESRX   
5  BBG000BVL406            UN  united_states_america               TRN   
6  BBG000BTGM43            UN  united_states_america              TREX   
7  BBG00KXXK940            UN  united_states_america                CI   
8  BBG00JGMWFQ5            UN  united_states_america               ACA   

  market_sector  security_type  coupon  
0        equity   common_stock     NaN  
1        equity   common_stock     NaN  
2          govt  us_government   2.000  
3          govt  us_government   6.875  
4        equity   common_stock     NaN  
5        equity   common_stock     NaN  
6        equity   common_stock     NaN  
7        equity   common_stock     NaN  
8        equity   common_stock     NaN

In [5]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}

In [6]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=ibor_scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

success  failed  errors
0        9       0       0

## Create LUSID Properties

Create the properties for the sub-holding keys and executed price on the allocations

In [7]:
properties_api = api_factory.build(la.PropertyDefinitionsApi)

def create_property_definition(domain, scope, code, data_type):
    try:
        properties_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain=domain,
                scope=scope,
                code=code,
                display_name=code,
                life_time="Perpetual",
                value_required=False,
                data_type_id=lm.resource_id.ResourceId(scope="system", code=data_type)
            )
        )
    except ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] != 124:  # 'PropertyAlreadyExists'
            raise e

In [8]:
# sub-holding keys
create_property_definition("Transaction", ibor_scope, strategy_shk, "string")

# allocation execution price
create_property_definition("Allocation", orders_scope, executed_price_code, "number")

# allocation strategy name
create_property_definition("Allocation", orders_scope, strategy_shk, "string")

## Upload a Portfolio into LUSID

In [9]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(data=[
        {"portfolio_code": portfolio_code, "portfolio_name": name},
    ])
    
    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$GBP",
        },
        "optional": {
            "created": f"${pf_created_date}"
        },
    }
    
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[strategy_shk],
    )

    succ, failed = format_portfolios_response(result)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])) 

Create the main portfolio

In [10]:
create_portfolio(ibor_scope, ibor_portfolio, "ibor")

success  failed  errors
0        1       0       0

## Create a Derived portfolio to hold intraday activity

Create a derived portfolio to capture the executed orders to give us a front office live postion view

In [11]:
derivedpf_api = la.DerivedTransactionPortfoliosApi(api_factory.build(la.DerivedTransactionPortfoliosApi))

try:
    dp_request = lm.CreateDerivedTransactionPortfolioRequest(
        code="us_long_live",
        display_name="us_long_live",
        parent_portfolio_id=lm.ResourceId(ibor_scope, ibor_portfolio),
        created=pf_created_date,    
    )

    dp_result = derivedpf_api.create_derived_portfolio(scope=ibor_scope, create_derived_transaction_portfolio_request=dp_request)
except ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 112:  # 'PortfolioWithIdAlreadyExists'
        raise e

## Set initial holdings in the parent portfolio

In [12]:
hldgs_df = pd.read_csv("data/initial_holdings.csv")
display(hldgs_df)

instrument_name client_internal          isin          figi  \
0     Amazon_Nasdaq_AMZN    imd_34634534  US0231351067  BBG000BVPXP1   
1      Apple_Nasdaq_AAPL    imd_35345345  US0378331005  BBG000B9XVV8   
2  USTreasury_6.875_2025    imd_34534539  US912810EV62  BBG000DQQNJ8   
3      Apple_Nasdaq_AAPL    imd_35345375  US0378331005  BBG000B9XVV8   
4   USTreasury_2.00_2021    imd_34535363  US912828U816  BBG00FN3B5K8   

   quantity  unit_cost   total_cost currency  strategy  
0      5000    1550.00   7750000.00      USD      Tech  
1     49567     190.00   9417730.00      USD      Tech  
2     98444     140.98  13878635.12      USD  Balanced  
3    350000     190.00   9417730.00      USD  Balanced  
4      1433      99.25  12063142.75      USD  Balanced

In [13]:
holdings_mapping = {
    "required":{
        "code": f"${ibor_portfolio}",
        "effective_at": "$2020-05-01",
        "tax_lots.units": "quantity"
    },
    "identifier_mapping": {
        "Figi": "figi"
    },
    "optional": {
        "tax_lots.cost.amount": "total_cost",
        "tax_lots.cost.currency": "currency",
        "tax_lots.price": "unit_cost"
    }
}

In [14]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=ibor_scope,
    data_frame=hldgs_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    identifier_mapping=holdings_mapping["identifier_mapping"],
    sub_holding_keys=[strategy_shk],
    file_type="holdings"
)

succ, failed = format_holdings_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

success  failed  errors
0        1       0       0

## Check the initial portfolio holdings

In [15]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings(scope=ibor_scope, code=ibor_portfolio, property_keys=["Instrument/default/Name"])

lusid_response_to_data_frame(executed_holdings, rename_properties=True).sort_values(by='strategy(middle_office-SubHoldingKeys)')

instrument_uid strategy(middle_office-SubHoldingKeys)  \
2  LUID_00003D53                               Balanced   
3  LUID_00003D4X                               Balanced   
4  LUID_00003D51                               Balanced   
0  LUID_00003D4V                                   Tech   
1  LUID_00003D53                                   Tech   

  Name(default-Properties) SourcePortfolioId(default-Properties)  \
2        Apple_Nasdaq_AAPL                               us_long   
3    USTreasury_6.875_2025                               us_long   
4     USTreasury_2.00_2021                               us_long   
0       Amazon_Nasdaq_AMZN                               us_long   
1        Apple_Nasdaq_AAPL                               us_long   

  SourcePortfolioScope(default-Properties) holding_type     units  \
2                            middle_office            P  350000.0   
3                            middle_office            P   98444.0   
4                            middle_office            P    1433.0   
0                            middle_office            P    5000.0   
1                            middle_office            P   49567.0   

   settled_units  cost.amount cost.currency  cost_portfolio_ccy.amount  \
2       350000.0   9417730.00           USD                        0.0   
3        98444.0  13878635.12           USD                        0.0   
4         1433.0  12063142.75           USD                        0.0   
0         5000.0   7750000.00           USD                        0.0   
1        49567.0   9417730.00           USD                        0.0   

  cost_portfolio_ccy.currency currency  
2                         GBP      USD  
3                         GBP      USD  
4                         GBP      USD  
0                         GBP      USD  
1                         GBP      USD

## Post orders into LUSID

In this section, we post some [orders](https://support.finbourne.com/how-does-lusid-support-the-trade-lifecycle) into LUSID. 

In [16]:
orders_df = pd.read_csv('data/orders.csv')
orders_df

portfolio        instrument_name client_internal          isin  \
0   us_long   USTreasury_2.00_2021    imd_34535347  US912828U816   
1   us_long   USTreasury_2.00_2021    imd_34535347  US912828U816   
2   us_long     Amazon_Nasdaq_AMZN   imd_346345343  US0231351067   
3   us_long     Amazon_Nasdaq_AMZN   imd_346345343  US0231351067   
4   us_long  USTreasury_6.875_2025    imd_34534512  US912810EV62   

           figi  quantity    price currency order_id   side   type  strategy  \
0  BBG00FN3B5K8    121543    99.25      USD   ORD001    buy  limit  Balanced   
1  BBG00FN3B5K8       121    99.30      USD   ORD002    buy  limit  Balanced   
2  BBG000BVPXP1       500  1400.00      USD   ORD003   sell  limit      Tech   
3  BBG000BVPXP1      4000  1550.00      USD   ORD004    buy  limit  Balanced   
4  BBG000DQQNJ8      5643   140.98      USD   ORD005  trade  limit  Balanced   

  state  
0   new  
1   new  
2   new  
3   new  
4   new

In [17]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)

for index, order in orders_df.iterrows():
    
    portfolio = order['portfolio']

    order_requests[portfolio].append(
        models.OrderRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/Figi': order['figi']
            },
            properties={},
            portfolio_id=models.ResourceId(
                scope=orders_scope,
                code=orders_portfolio
            )
        )
    )

for order_portfolio in order_requests:
    order_sets[order_portfolio] = models.OrderSetRequest(
        order_requests=order_requests[order_portfolio]
    )    

for order_portfolio in order_sets:    
    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=order_sets[order_portfolio]
    )
    
    display(lusid_response_to_data_frame(response))

properties     version.effective_from               version.as_at_date  \
0         {}  0001-01-01 00:00:00+00:00 2021-10-13 11:06:04.343330+00:00   
1         {}  0001-01-01 00:00:00+00:00 2021-10-13 11:06:04.343330+00:00   
2         {}  0001-01-01 00:00:00+00:00 2021-10-13 11:06:04.343330+00:00   
3         {}  0001-01-01 00:00:00+00:00 2021-10-13 11:06:04.343330+00:00   
4         {}  0001-01-01 00:00:00+00:00 2021-10-13 11:06:04.343330+00:00   

  instrument_identifiers.Instrument/default/Figi  quantity   side  \
0                                   BBG000DQQNJ8      5643  trade   
1                                   BBG000BVPXP1      4000    buy   
2                                   BBG00FN3B5K8       121    buy   
3                                   BBG000BVPXP1       500   sell   
4                                   BBG00FN3B5K8    121543    buy   

  order_book_id.scope order_book_id.code         portfolio_id.scope  \
0        UnknownScope   UnknownOrderBook  combining_fo_and_mo_scope   
1        UnknownScope   UnknownOrderBook  combining_fo_and_mo_scope   
2        UnknownScope   UnknownOrderBook  combining_fo_and_mo_scope   
3        UnknownScope   UnknownOrderBook  combining_fo_and_mo_scope   
4        UnknownScope   UnknownOrderBook  combining_fo_and_mo_scope   

  portfolio_id.code                   id.scope id.code lusid_instrument_id  \
0      us_long_live  combining_fo_and_mo_scope  ORD005       LUID_00003D4X   
1      us_long_live  combining_fo_and_mo_scope  ORD004       LUID_00003D4V   
2      us_long_live  combining_fo_and_mo_scope  ORD002       LUID_00003D51   
3      us_long_live  combining_fo_and_mo_scope  ORD003       LUID_00003D4V   
4      us_long_live  combining_fo_and_mo_scope  ORD001       LUID_00003D51   

                              date  price.amount price.currency  \
0 2021-10-13 11:06:04.330994+00:00           0.0            ZZZ   
1 2021-10-13 11:06:04.330966+00:00           0.0            ZZZ   
2 2021-10-13 11:06:04.330908+00:00           0.0            ZZZ   
3 2021-10-13 11:06:04.330938+00:00           0.0            ZZZ   
4 2021-10-13 11:06:04.330837+00:00           0.0            ZZZ   

  order_instruction_id.scope order_instruction_id.code package_id.scope  \
0               UnknownScope   UnknownOrderInstruction     UnknownScope   
1               UnknownScope   UnknownOrderInstruction     UnknownScope   
2               UnknownScope   UnknownOrderInstruction     UnknownScope   
3               UnknownScope   UnknownOrderInstruction     UnknownScope   
4               UnknownScope   UnknownOrderInstruction     UnknownScope   

  package_id.code  
0  UnknownPackage  
1  UnknownPackage  
2  UnknownPackage  
3  UnknownPackage  
4  UnknownPackage

## Book Allocations against the orders

[Allocations](https://support.finbourne.com/what-is-an-allocation-in-lusid) are then booked off the back of these orders.

In [18]:
allocations_df = pd.read_csv('data/allocations.csv')
allocations_df

portfolio        instrument_name client_internal          isin  \
0   us_long   USTreasury_2.00_2021    imd_34535347  US912828U816   
1   us_long     Amazon_Nasdaq_AMZN   imd_346345343  US0231351067   
2   us_long     Amazon_Nasdaq_AMZN   imd_346345343  US0231351067   
3   us_long  USTreasury_6.875_2025    imd_34534512  US912810EV62   
4   us_long  USTreasury_6.875_2025    imd_34534539  US912810EV62   

           figi  quantity  price currency allocation_id originating_order  \
0  BBG00FN3B5K8    121543     80      USD      ALLOC001            ORD001   
1  BBG000BVPXP1       500   1600      USD      ALLOC002            ORD002   
2  BBG000BVPXP1      4000   1600      USD      ALLOC003            ORD003   
3  BBG000DQQNJ8      5643    150      USD      ALLOC004            ORD004   
4  BBG000DQQNJ8     32000    150      USD      ALLOC005            ORD005   

  state  side  strategy  
0   new   buy  Balanced  
1   new  sell      Tech  
2   new   buy  Balanced  
3   new   buy  Balanced  
4   new  sell  Balanced

In [19]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)

for index, allocation in allocations_df.iterrows():
    
    portfolio = allocation['portfolio']

    allocation_requests[portfolio].append(
        models.AllocationRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=allocation['allocation_id']
            ),
            allocated_order_id=models.ResourceId(
                scope=orders_scope,
                code=allocation['originating_order']
            ),
            quantity=allocation['quantity'],
            instrument_identifiers={
                'Instrument/default/Figi': allocation['figi']
            },
            properties={
                execution_price_property: lm.PerpetualProperty(
                    key=execution_price_property,
                    value=lm.PropertyValue(metric_value=lm.MetricValue(value=allocation["price"]))
                ),
                allocation_strategy_property: lm.PerpetualProperty(
                    key=allocation_strategy_property,
                    value=lm.PropertyValue(label_value=allocation['strategy'])
                )
            },
            portfolio_id=models.ResourceId(
                scope=orders_scope,
                code=orders_portfolio
            )
        )
    )
    
for allocation_portfolio in allocation_requests:    
    allocation_sets[allocation_portfolio] = models.AllocationSetRequest(
        allocation_requests=allocation_requests[allocation_portfolio]
    )

for allocation_portfolio in allocation_sets:
    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        allocation_set_request=allocation_sets[allocation_portfolio]
    )
    
    display(lusid_response_to_data_frame(response))

id.scope   id.code   allocated_order_id.scope  \
0  combining_fo_and_mo_scope  ALLOC002  combining_fo_and_mo_scope   
1  combining_fo_and_mo_scope  ALLOC005  combining_fo_and_mo_scope   
2  combining_fo_and_mo_scope  ALLOC004  combining_fo_and_mo_scope   
3  combining_fo_and_mo_scope  ALLOC003  combining_fo_and_mo_scope   
4  combining_fo_and_mo_scope  ALLOC001  combining_fo_and_mo_scope   

  allocated_order_id.code         portfolio_id.scope portfolio_id.code  \
0                  ORD002  combining_fo_and_mo_scope      us_long_live   
1                  ORD005  combining_fo_and_mo_scope      us_long_live   
2                  ORD004  combining_fo_and_mo_scope      us_long_live   
3                  ORD003  combining_fo_and_mo_scope      us_long_live   
4                  ORD001  combining_fo_and_mo_scope      us_long_live   

   quantity instrument_identifiers.Instrument/default/Figi  \
0       500                                   BBG000BVPXP1   
1     32000                                   BBG000DQQNJ8   
2      5643                                   BBG000DQQNJ8   
3      4000                                   BBG000BVPXP1   
4    121543                                   BBG00FN3B5K8   

      version.effective_from               version.as_at_date  \
0  0001-01-01 00:00:00+00:00 2021-10-13 11:06:04.433216+00:00   
1  0001-01-01 00:00:00+00:00 2021-10-13 11:06:04.433216+00:00   
2  0001-01-01 00:00:00+00:00 2021-10-13 11:06:04.433216+00:00   
3  0001-01-01 00:00:00+00:00 2021-10-13 11:06:04.433216+00:00   
4  0001-01-01 00:00:00+00:00 2021-10-13 11:06:04.433216+00:00   

  properties.Allocation/combining_fo_and_mo_scope/executed_price.key  \
0  Allocation/combining_fo_and_mo_scope/executed_...                   
1  Allocation/combining_fo_and_mo_scope/executed_...                   
2  Allocation/combining_fo_and_mo_scope/executed_...                   
3  Allocation/combining_fo_and_mo_scope/executed_...                   
4  Allocation/combining_fo_and_mo_scope/executed_...                   

   properties.Allocation/combining_fo_and_mo_scope/executed_price.value.metric_value.value  \
0                                             1600.0                                         
1                                              150.0                                         
2                                              150.0                                         
3                                             1600.0                                         
4                                               80.0                                         

  properties.Allocation/combining_fo_and_mo_scope/strategy.key  \
0      Allocation/combining_fo_and_mo_scope/strategy             
1      Allocation/combining_fo_and_mo_scope/strategy             
2      Allocation/combining_fo_and_mo_scope/strategy             
3      Allocation/combining_fo_and_mo_scope/strategy             
4      Allocation/combining_fo_and_mo_scope/strategy             

  properties.Allocation/combining_fo_and_mo_scope/strategy.value.label_value  \
0                                               Tech                           
1                                           Balanced                           
2                                           Balanced                           
3                                           Balanced                           
4                                           Balanced                           

  lusid_instrument_id placement_ids         state         side         type  \
0       LUID_00003D4V            []  UnknownState  UnknownSide  UnknownType   
1       LUID_00003D4X            []  UnknownState  UnknownSide  UnknownType   
2       LUID_00003D4X            []  UnknownState  UnknownSide  UnknownType   
3       LUID_00003D4V            []  UnknownState  UnknownSide  UnknownType   
4       LUID_00003D51            []  UnknownState  UnknownSide  UnknownType   

             settlement_date                 

##  Post transactions for the allocations

In [20]:
# fetch orders
orders_response = api_factory.build(lusid.api.OrdersApi).list_orders(filter="Id.Scope eq '" + orders_scope + "'")

# fetch allocations
allocations_response = api_factory.build(lusid.api.AllocationsApi).list_allocations(filter="Id.Scope eq '" + orders_scope + "'")

In [21]:
order_responses = {}
allocation_responses = {} 

for order in orders_response.values:
    order_responses[order.id.code] = order

for allocation in allocations_response.values:
    allocation_responses[allocation.allocated_order_id.code] = allocation
    
batch_transaction_requests = defaultdict(list)
strategy_property = f"Transaction/{ibor_scope}/{strategy_shk}"

for order in order_responses:
    
    allocation = allocation_responses[order]
    
    type = "Buy" if order_responses[order].side.lower() == "buy" else "Sell"
    
    executed_price = allocation.properties[execution_price_property].value.metric_value.value
    
    batch_transaction_requests[order_responses[order].portfolio_id.code].append(
        models.TransactionRequest(
            transaction_id = 'TR.' + allocation.id.code,
            type = type,
            instrument_identifiers = allocation.instrument_identifiers,
            transaction_date = '2020-05-02',
            settlement_date = '2020-05-04',
            units = allocation.quantity,
            transaction_price = models.TransactionPrice(
                price = executed_price,
                type = 'Price'),
            total_consideration = models.CurrencyAndAmount(
                amount = allocation.quantity * executed_price,
                currency = 'USD'
            ),
            transaction_currency = 'USD',
            properties={
                strategy_property: lm.PerpetualProperty(
                    key=strategy_property,
                    value=lm.PropertyValue(
                        label_value=allocation.properties[allocation_strategy_property].value.label_value
                    )
                )
            }
        )
    )

In [22]:
for portfolio_name in batch_transaction_requests:
    
    transaction_response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
        scope=ibor_scope,
        code=orders_portfolio,
        transaction_request=batch_transaction_requests[portfolio_name])

## Check positions adjusted by intraday orders

In [23]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings(scope=ibor_scope, code=orders_portfolio, property_keys=["Instrument/default/Name"])

lusid_response_to_data_frame(executed_holdings, rename_properties=True).sort_values(by='strategy(middle_office-SubHoldingKeys)')

instrument_uid strategy(middle_office-SubHoldingKeys)  \
1  LUID_00003D4V                               Balanced   
3  LUID_00003D53                               Balanced   
4  LUID_00003D4X                               Balanced   
5  LUID_00003D51                               Balanced   
6        CCY_USD                               Balanced   
0  LUID_00003D4V                                   Tech   
2  LUID_00003D53                                   Tech   
7        CCY_USD                                   Tech   

  Name(default-Properties) SourcePortfolioId(default-Properties)  \
1       Amazon_Nasdaq_AMZN                          us_long_live   
3        Apple_Nasdaq_AAPL                          us_long_live   
4    USTreasury_6.875_2025                          us_long_live   
5     USTreasury_2.00_2021                          us_long_live   
6     UNITED STATES DOLLAR                          us_long_live   
0       Amazon_Nasdaq_AMZN                          us_long_live   
2        Apple_Nasdaq_AAPL                          us_long_live   
7     UNITED STATES DOLLAR                          us_long_live   

  SourcePortfolioScope(default-Properties) holding_type     units  \
1                            middle_office            P   -4000.0   
3                            middle_office            P  350000.0   
4                            middle_office            P   72087.0   
5                            middle_office            P  122976.0   
6                            middle_office            B  630110.0   
0                            middle_office            P    5500.0   
2                            middle_office            P   49567.0   
7                            middle_office            B -800000.0   

   settled_units  cost.amount cost.currency  cost_portfolio_ccy.amount  \
1        -4000.0  -6400000.00           USD                        0.0   
3       350000.0   9417730.00           USD                        0.0   
4        72087.0  10198076.72           USD                        0.0   
5       122976.0  21786582.75           USD                        0.0   
6       630110.0    630110.00           USD                        0.0   
0         5500.0   8550000.00           USD                        0.0   
2        49567.0   9417730.00           USD                        0.0   
7      -800000.0   -800000.00           USD                        0.0   

  cost_portfolio_ccy.currency currency  
1                         GBP      USD  
3                         GBP      USD  
4                         GBP      USD  
5                         GBP      USD  
6                         GBP      USD  
0                         GBP      USD  
2                         GBP      USD  
7                         GBP      USD